# Synchronize Annif corpus files against latest YSO

This notebook can be used to update Annif corpus files that have been created using an older version of YSO. It will read the old corpus files, look up YSO URIs in the new YSO version, update their labels in case they have changed, change deprecated URIs to their successors and remove any URIs that are not in the new YSO (should not happen but...).

Requirements:

* jupyter (obviously)
* rdflib

In [2]:
# Parse the current version of YSO

import rdflib
yso = rdflib.Graph()
yso.parse('../vocab/yso-skos.ttl', format='turtle')
print(len(yso))  # show number of triples

692303


In [3]:
# Define helper functions

import functools
import glob
from rdflib.namespace import RDF, OWL, DCTERMS, SKOS

def parse_tsv(fn):
    with open(fn) as tsvf:
        for line in tsvf:
            line = line.strip()
            if '\t' not in line:
                print("warning: cannot parse line, skipping:", line)
                continue
            uri = uri[1:-1]
            yield (uri, label)

@functools.lru_cache(maxsize=30000)
def concept_label(uri, default, lang):
    labels = yso.preferredLabel(uri, lang=lang)
    if labels:
        return str(labels[0][1])
    else:
        return default

@functools.lru_cache(maxsize=30000)
def replace_concept(subj, lang):
    uri, label = subj
    uri = rdflib.URIRef(uri)
    replacements = list(yso.objects(uri, DCTERMS.isReplacedBy))
    if len(replacements) == 0:
        return [(str(uri), concept_label(uri, label, lang))]
    else:
        print("replacing", uri, "with", ' '.join([str(r) for r in replacements]))
        if len(replacements) > 1:
            print("warning: multiple replacements for", uri)
        return [(str(r), concept_label(r, label, lang)) for r in replacements]

@functools.lru_cache(maxsize=30000)
def check_concept(uri, lang):
    uri = rdflib.URIRef(uri)
    if (uri, RDF.type, SKOS.Concept) not in yso:
        print(uri, "not a skos:Concept")
        return False
    if (uri, OWL.deprecated, True) in yso:
        print(uri, "is deprecated")
        return False
    if concept_label(uri, None, lang) is None:
        print(uri, "has no label")
        return False
    return True

def write_tsv(fn, subjects):
    with open(fn, 'w') as tsvf:
        for uri, label in subjects:
            print("<{}>\t{}".format(uri, label), file=tsvf)

def sync_fulltext_dir(pattern, lang):
    for idx, fn in enumerate(glob.glob(pattern)):
        print(fn)
        l = list(parse_tsv(fn))
        #print(l)
        subjects = []
        for subj in l:
            subjects += replace_concept(subj, lang)
        for uri, label in subjects:
            check_concept(uri, lang)
        #print(subjects)
        write_tsv(fn, subjects)


In [4]:
sync_fulltext_dir('../fulltext/kirjastonhoitaja/*/*.tsv', 'fi')

../fulltext/kirjastonhoitaja/train/415610.tsv
../fulltext/kirjastonhoitaja/train/432388.tsv
../fulltext/kirjastonhoitaja/train/404326.tsv
../fulltext/kirjastonhoitaja/train/421535.tsv
../fulltext/kirjastonhoitaja/train/412377.tsv
../fulltext/kirjastonhoitaja/train/414829.tsv
../fulltext/kirjastonhoitaja/train/409917.tsv
../fulltext/kirjastonhoitaja/train/436502.tsv
../fulltext/kirjastonhoitaja/train/422936.tsv
../fulltext/kirjastonhoitaja/train/431040.tsv
../fulltext/kirjastonhoitaja/train/396342.tsv
../fulltext/kirjastonhoitaja/train/391321.tsv
../fulltext/kirjastonhoitaja/train/408518.tsv
../fulltext/kirjastonhoitaja/train/397045.tsv
../fulltext/kirjastonhoitaja/train/423902.tsv
../fulltext/kirjastonhoitaja/train/421287.tsv
../fulltext/kirjastonhoitaja/train/388971.tsv
../fulltext/kirjastonhoitaja/train/386149.tsv
../fulltext/kirjastonhoitaja/train/398086.tsv
../fulltext/kirjastonhoitaja/train/403375.tsv
../fulltext/kirjastonhoitaja/train/385215.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/407042.tsv
../fulltext/kirjastonhoitaja/train/418068.tsv
../fulltext/kirjastonhoitaja/train/435301.tsv
../fulltext/kirjastonhoitaja/train/418914.tsv
../fulltext/kirjastonhoitaja/train/386420.tsv
../fulltext/kirjastonhoitaja/train/429822.tsv
../fulltext/kirjastonhoitaja/train/427782.tsv
../fulltext/kirjastonhoitaja/train/434479.tsv
../fulltext/kirjastonhoitaja/train/421071.tsv
../fulltext/kirjastonhoitaja/train/384687.tsv
../fulltext/kirjastonhoitaja/train/389847.tsv
../fulltext/kirjastonhoitaja/train/384810.tsv
../fulltext/kirjastonhoitaja/train/411664.tsv
../fulltext/kirjastonhoitaja/train/423527.tsv
../fulltext/kirjastonhoitaja/train/392206.tsv
../fulltext/kirjastonhoitaja/train/419421.tsv
../fulltext/kirjastonhoitaja/train/428635.tsv
../fulltext/kirjastonhoitaja/train/420844.tsv
../fulltext/kirjastonhoitaja/train/434467.tsv
../fulltext/kirjastonhoitaja/train/424456.tsv
../fulltext/kirjastonhoitaja/train/388892.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/394376.tsv
../fulltext/kirjastonhoitaja/train/395885.tsv
../fulltext/kirjastonhoitaja/train/425976.tsv
../fulltext/kirjastonhoitaja/train/393497.tsv
../fulltext/kirjastonhoitaja/train/420386.tsv
../fulltext/kirjastonhoitaja/train/425674.tsv
../fulltext/kirjastonhoitaja/train/428059.tsv
../fulltext/kirjastonhoitaja/train/435268.tsv
../fulltext/kirjastonhoitaja/train/425926.tsv
../fulltext/kirjastonhoitaja/train/403430.tsv
../fulltext/kirjastonhoitaja/train/393874.tsv
../fulltext/kirjastonhoitaja/train/435555.tsv
../fulltext/kirjastonhoitaja/train/390031.tsv
../fulltext/kirjastonhoitaja/train/387368.tsv
../fulltext/kirjastonhoitaja/train/418784.tsv
../fulltext/kirjastonhoitaja/train/402425.tsv
../fulltext/kirjastonhoitaja/train/434148.tsv
../fulltext/kirjastonhoitaja/train/386030.tsv
../fulltext/kirjastonhoitaja/train/387197.tsv
../fulltext/kirjastonhoitaja/train/412817.tsv
../fulltext/kirjastonhoitaja/train/421844.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/421418.tsv
../fulltext/kirjastonhoitaja/train/432978.tsv
../fulltext/kirjastonhoitaja/train/406467.tsv
../fulltext/kirjastonhoitaja/train/436606.tsv
../fulltext/kirjastonhoitaja/train/420737.tsv
../fulltext/kirjastonhoitaja/train/386411.tsv
../fulltext/kirjastonhoitaja/train/428761.tsv
../fulltext/kirjastonhoitaja/train/427972.tsv
../fulltext/kirjastonhoitaja/train/395169.tsv
../fulltext/kirjastonhoitaja/train/385445.tsv
../fulltext/kirjastonhoitaja/train/429351.tsv
../fulltext/kirjastonhoitaja/train/401149.tsv
../fulltext/kirjastonhoitaja/train/402557.tsv
../fulltext/kirjastonhoitaja/train/395032.tsv
../fulltext/kirjastonhoitaja/train/397922.tsv
../fulltext/kirjastonhoitaja/train/436676.tsv
../fulltext/kirjastonhoitaja/train/398660.tsv
../fulltext/kirjastonhoitaja/train/428460.tsv
../fulltext/kirjastonhoitaja/train/404991.tsv
../fulltext/kirjastonhoitaja/train/417145.tsv
../fulltext/kirjastonhoitaja/train/436715.tsv
../fulltext/kirjastonhoitaja/train

../fulltext/kirjastonhoitaja/train/434991.tsv
../fulltext/kirjastonhoitaja/train/408618.tsv
../fulltext/kirjastonhoitaja/train/420410.tsv
../fulltext/kirjastonhoitaja/train/394518.tsv
../fulltext/kirjastonhoitaja/train/434885.tsv
../fulltext/kirjastonhoitaja/train/419175.tsv
../fulltext/kirjastonhoitaja/train/434879.tsv
../fulltext/kirjastonhoitaja/train/436189.tsv
../fulltext/kirjastonhoitaja/train/436481.tsv
../fulltext/kirjastonhoitaja/train/422832.tsv
../fulltext/kirjastonhoitaja/train/410788.tsv
../fulltext/kirjastonhoitaja/train/436523.tsv
../fulltext/kirjastonhoitaja/train/435088.tsv
../fulltext/kirjastonhoitaja/train/410041.tsv
../fulltext/kirjastonhoitaja/train/428433.tsv
../fulltext/kirjastonhoitaja/train/417280.tsv
../fulltext/kirjastonhoitaja/train/431643.tsv
../fulltext/kirjastonhoitaja/train/417788.tsv
../fulltext/kirjastonhoitaja/train/425771.tsv
../fulltext/kirjastonhoitaja/train/405541.tsv
../fulltext/kirjastonhoitaja/train/432375.tsv
../fulltext/kirjastonhoitaja/train

...

../fulltext/kirjastonhoitaja/all/439259.tsv
../fulltext/kirjastonhoitaja/all/403292.tsv
../fulltext/kirjastonhoitaja/all/437711.tsv
../fulltext/kirjastonhoitaja/all/437428.tsv
../fulltext/kirjastonhoitaja/all/424299.tsv
../fulltext/kirjastonhoitaja/all/435843.tsv
../fulltext/kirjastonhoitaja/all/394457.tsv
../fulltext/kirjastonhoitaja/all/431180.tsv
../fulltext/kirjastonhoitaja/all/425950.tsv
../fulltext/kirjastonhoitaja/all/415684.tsv
../fulltext/kirjastonhoitaja/all/390913.tsv
../fulltext/kirjastonhoitaja/all/440518.tsv
../fulltext/kirjastonhoitaja/all/434579.tsv
../fulltext/kirjastonhoitaja/all/392088.tsv
../fulltext/kirjastonhoitaja/all/406922.tsv
../fulltext/kirjastonhoitaja/all/388260.tsv
../fulltext/kirjastonhoitaja/all/401974.tsv
../fulltext/kirjastonhoitaja/all/389720.tsv
../fulltext/kirjastonhoitaja/all/409375.tsv
../fulltext/kirjastonhoitaja/all/420742.tsv
../fulltext/kirjastonhoitaja/all/419111.tsv
../fulltext/kirjastonhoitaja/all/420770.tsv
../fulltext/kirjastonhoitaja/all

../fulltext/kirjastonhoitaja/all/432592.tsv
../fulltext/kirjastonhoitaja/all/434064.tsv
../fulltext/kirjastonhoitaja/all/437660.tsv
../fulltext/kirjastonhoitaja/all/398850.tsv
../fulltext/kirjastonhoitaja/all/385671.tsv
../fulltext/kirjastonhoitaja/all/408412.tsv
../fulltext/kirjastonhoitaja/all/419126.tsv
../fulltext/kirjastonhoitaja/all/401646.tsv
../fulltext/kirjastonhoitaja/all/434155.tsv
../fulltext/kirjastonhoitaja/all/436765.tsv
../fulltext/kirjastonhoitaja/all/399074.tsv
../fulltext/kirjastonhoitaja/all/424554.tsv
../fulltext/kirjastonhoitaja/all/431029.tsv
../fulltext/kirjastonhoitaja/all/433894.tsv
../fulltext/kirjastonhoitaja/all/440548.tsv
../fulltext/kirjastonhoitaja/all/405323.tsv
../fulltext/kirjastonhoitaja/all/418449.tsv
../fulltext/kirjastonhoitaja/all/423652.tsv
../fulltext/kirjastonhoitaja/all/417812.tsv
../fulltext/kirjastonhoitaja/all/385528.tsv
../fulltext/kirjastonhoitaja/all/424694.tsv
../fulltext/kirjastonhoitaja/all/424111.tsv
../fulltext/kirjastonhoitaja/all

../fulltext/kirjastonhoitaja/all/416629.tsv
../fulltext/kirjastonhoitaja/all/419093.tsv
../fulltext/kirjastonhoitaja/all/431883.tsv
../fulltext/kirjastonhoitaja/all/423570.tsv
../fulltext/kirjastonhoitaja/all/433475.tsv
../fulltext/kirjastonhoitaja/all/443190.tsv
../fulltext/kirjastonhoitaja/all/429022.tsv
../fulltext/kirjastonhoitaja/all/435838.tsv
../fulltext/kirjastonhoitaja/all/438543.tsv
../fulltext/kirjastonhoitaja/all/424560.tsv
../fulltext/kirjastonhoitaja/all/394033.tsv
../fulltext/kirjastonhoitaja/all/437964.tsv
../fulltext/kirjastonhoitaja/all/435375.tsv
../fulltext/kirjastonhoitaja/all/441353.tsv
../fulltext/kirjastonhoitaja/all/386158.tsv
../fulltext/kirjastonhoitaja/all/430350.tsv
../fulltext/kirjastonhoitaja/all/437467.tsv
../fulltext/kirjastonhoitaja/all/426414.tsv
../fulltext/kirjastonhoitaja/all/416318.tsv
../fulltext/kirjastonhoitaja/all/404798.tsv
../fulltext/kirjastonhoitaja/all/386489.tsv
../fulltext/kirjastonhoitaja/all/424424.tsv
../fulltext/kirjastonhoitaja/all

../fulltext/kirjastonhoitaja/all/386287.tsv
../fulltext/kirjastonhoitaja/all/439155.tsv
../fulltext/kirjastonhoitaja/all/438333.tsv
../fulltext/kirjastonhoitaja/all/435590.tsv
../fulltext/kirjastonhoitaja/all/441420.tsv
../fulltext/kirjastonhoitaja/all/396845.tsv
../fulltext/kirjastonhoitaja/all/435224.tsv
../fulltext/kirjastonhoitaja/all/407601.tsv
../fulltext/kirjastonhoitaja/all/418714.tsv
../fulltext/kirjastonhoitaja/all/429114.tsv
../fulltext/kirjastonhoitaja/all/443708.tsv
../fulltext/kirjastonhoitaja/all/409292.tsv
../fulltext/kirjastonhoitaja/all/441281.tsv
../fulltext/kirjastonhoitaja/all/387488.tsv
../fulltext/kirjastonhoitaja/all/412677.tsv
../fulltext/kirjastonhoitaja/all/407024.tsv
../fulltext/kirjastonhoitaja/all/412808.tsv
../fulltext/kirjastonhoitaja/all/423437.tsv
../fulltext/kirjastonhoitaja/all/427039.tsv
../fulltext/kirjastonhoitaja/all/416646.tsv
../fulltext/kirjastonhoitaja/all/430107.tsv
../fulltext/kirjastonhoitaja/all/407929.tsv
../fulltext/kirjastonhoitaja/all

../fulltext/kirjastonhoitaja/test/443637.tsv
../fulltext/kirjastonhoitaja/test/441619.tsv
../fulltext/kirjastonhoitaja/test/440614.tsv
../fulltext/kirjastonhoitaja/test/442176.tsv
../fulltext/kirjastonhoitaja/test/443406.tsv
../fulltext/kirjastonhoitaja/test/441425.tsv
../fulltext/kirjastonhoitaja/test/443483.tsv
../fulltext/kirjastonhoitaja/test/440548.tsv
../fulltext/kirjastonhoitaja/test/443672.tsv
../fulltext/kirjastonhoitaja/test/442996.tsv
../fulltext/kirjastonhoitaja/test/440636.tsv
../fulltext/kirjastonhoitaja/test/442225.tsv
../fulltext/kirjastonhoitaja/test/443038.tsv
../fulltext/kirjastonhoitaja/test/441955.tsv
../fulltext/kirjastonhoitaja/test/439560.tsv
../fulltext/kirjastonhoitaja/test/441398.tsv
../fulltext/kirjastonhoitaja/test/443287.tsv
../fulltext/kirjastonhoitaja/test/441864.tsv
../fulltext/kirjastonhoitaja/test/441365.tsv
../fulltext/kirjastonhoitaja/test/440722.tsv
../fulltext/kirjastonhoitaja/test/443713.tsv
../fulltext/kirjastonhoitaja/test/439954.tsv
../fulltex

In [5]:
sync_fulltext_dir('../fulltext/jyu-theses/fin/*.tsv', 'fi')

../fulltext/jyu-theses/fin/2014-M-43582.tsv
../fulltext/jyu-theses/fin/2010-M-23254.tsv
../fulltext/jyu-theses/fin/2016-M-51853.tsv
../fulltext/jyu-theses/fin/2016-D-49352.tsv
../fulltext/jyu-theses/fin/2014-M-44648.tsv
../fulltext/jyu-theses/fin/2017-M-54574.tsv
../fulltext/jyu-theses/fin/2010-M-26787.tsv
../fulltext/jyu-theses/fin/2017-M-56168.tsv
../fulltext/jyu-theses/fin/2015-M-47285.tsv
../fulltext/jyu-theses/fin/2015-D-46762.tsv
../fulltext/jyu-theses/fin/2014-M-45335.tsv
../fulltext/jyu-theses/fin/2017-M-52947.tsv
../fulltext/jyu-theses/fin/2014-M-43639.tsv
../fulltext/jyu-theses/fin/2017-M-55761.tsv
../fulltext/jyu-theses/fin/2013-M-42143.tsv
../fulltext/jyu-theses/fin/2016-M-51641.tsv
../fulltext/jyu-theses/fin/2011-M-37926.tsv
../fulltext/jyu-theses/fin/2014-M-44985.tsv
../fulltext/jyu-theses/fin/2016-M-49846.tsv
../fulltext/jyu-theses/fin/2013-M-42305.tsv
../fulltext/jyu-theses/fin/2011-M-26709.tsv
../fulltext/jyu-theses/fin/2013-M-42628.tsv
../fulltext/jyu-theses/fin/2017-

../fulltext/jyu-theses/fin/2010-M-25954.tsv
../fulltext/jyu-theses/fin/2014-M-45001.tsv
../fulltext/jyu-theses/fin/2011-M-27139.tsv
../fulltext/jyu-theses/fin/2012-M-37955.tsv
../fulltext/jyu-theses/fin/2011-M-36575.tsv
../fulltext/jyu-theses/fin/2013-M-42494.tsv
../fulltext/jyu-theses/fin/2016-M-51835.tsv
../fulltext/jyu-theses/fin/2012-M-37893.tsv
../fulltext/jyu-theses/fin/2012-M-40905.tsv
../fulltext/jyu-theses/fin/2017-M-54575.tsv
../fulltext/jyu-theses/fin/2016-D-51215.tsv
../fulltext/jyu-theses/fin/2010-M-25650.tsv
../fulltext/jyu-theses/fin/2017-M-53634.tsv
../fulltext/jyu-theses/fin/2016-M-49676.tsv
../fulltext/jyu-theses/fin/2013-M-41911.tsv
../fulltext/jyu-theses/fin/2015-M-46176.tsv
../fulltext/jyu-theses/fin/2014-M-43916.tsv
../fulltext/jyu-theses/fin/2015-M-46141.tsv
../fulltext/jyu-theses/fin/2010-M-23766.tsv
../fulltext/jyu-theses/fin/2017-M-56078.tsv
../fulltext/jyu-theses/fin/2016-M-50301.tsv
../fulltext/jyu-theses/fin/2012-M-38282.tsv
../fulltext/jyu-theses/fin/2015-

../fulltext/jyu-theses/fin/2016-M-52452.tsv
../fulltext/jyu-theses/fin/2014-M-43339.tsv
../fulltext/jyu-theses/fin/2010-M-23427.tsv
../fulltext/jyu-theses/fin/2016-M-48579.tsv
../fulltext/jyu-theses/fin/2015-M-48126.tsv
../fulltext/jyu-theses/fin/2013-M-42391.tsv
../fulltext/jyu-theses/fin/2013-M-41571.tsv
../fulltext/jyu-theses/fin/2012-M-40551.tsv
../fulltext/jyu-theses/fin/2017-M-56754.tsv
../fulltext/jyu-theses/fin/2016-M-49299.tsv
../fulltext/jyu-theses/fin/2010-M-25755.tsv
../fulltext/jyu-theses/fin/2012-M-40000.tsv
../fulltext/jyu-theses/fin/2015-M-46644.tsv
../fulltext/jyu-theses/fin/2014-M-43208.tsv
../fulltext/jyu-theses/fin/2014-M-45007.tsv
../fulltext/jyu-theses/fin/2011-M-37312.tsv
../fulltext/jyu-theses/fin/2013-M-41927.tsv
../fulltext/jyu-theses/fin/2014-D-42803.tsv
../fulltext/jyu-theses/fin/2013-M-41855.tsv
../fulltext/jyu-theses/fin/2010-M-25704.tsv
../fulltext/jyu-theses/fin/2013-M-42491.tsv
../fulltext/jyu-theses/fin/2010-M-25789.tsv
../fulltext/jyu-theses/fin/2015-

../fulltext/jyu-theses/fin/2017-M-54141.tsv
../fulltext/jyu-theses/fin/2012-M-40620.tsv
../fulltext/jyu-theses/fin/2012-M-37890.tsv
../fulltext/jyu-theses/fin/2017-M-54547.tsv
../fulltext/jyu-theses/fin/2012-M-40188.tsv
../fulltext/jyu-theses/fin/2015-D-45843.tsv
../fulltext/jyu-theses/fin/2014-M-44680.tsv
../fulltext/jyu-theses/fin/2012-M-37290.tsv
../fulltext/jyu-theses/fin/2014-M-42932.tsv
../fulltext/jyu-theses/fin/2011-M-37168.tsv
../fulltext/jyu-theses/fin/2017-M-56462.tsv
../fulltext/jyu-theses/fin/2015-M-47972.tsv
../fulltext/jyu-theses/fin/2017-M-56453.tsv
../fulltext/jyu-theses/fin/2011-M-27222.tsv
../fulltext/jyu-theses/fin/2011-M-36518.tsv
../fulltext/jyu-theses/fin/2013-M-42559.tsv
../fulltext/jyu-theses/fin/2016-M-49007.tsv
../fulltext/jyu-theses/fin/2017-M-54387.tsv
../fulltext/jyu-theses/fin/2014-M-43697.tsv
../fulltext/jyu-theses/fin/2011-M-36721.tsv
../fulltext/jyu-theses/fin/2010-M-23030.tsv
../fulltext/jyu-theses/fin/2015-M-47807.tsv
../fulltext/jyu-theses/fin/2014-

../fulltext/jyu-theses/fin/2013-M-40986.tsv
../fulltext/jyu-theses/fin/2014-M-43187.tsv
../fulltext/jyu-theses/fin/2011-M-36816.tsv
../fulltext/jyu-theses/fin/2017-M-56907.tsv
../fulltext/jyu-theses/fin/2013-M-41082.tsv
../fulltext/jyu-theses/fin/2012-M-40458.tsv
../fulltext/jyu-theses/fin/2013-M-42609.tsv
../fulltext/jyu-theses/fin/2016-M-50916.tsv
../fulltext/jyu-theses/fin/2016-M-49537.tsv
../fulltext/jyu-theses/fin/2015-M-45725.tsv
../fulltext/jyu-theses/fin/2014-M-43699.tsv
../fulltext/jyu-theses/fin/2011-M-27001.tsv
../fulltext/jyu-theses/fin/2016-M-51391.tsv
../fulltext/jyu-theses/fin/2010-M-25667.tsv
../fulltext/jyu-theses/fin/2016-M-50449.tsv
../fulltext/jyu-theses/fin/2014-M-45038.tsv
../fulltext/jyu-theses/fin/2015-D-46576.tsv
../fulltext/jyu-theses/fin/2011-M-37351.tsv
../fulltext/jyu-theses/fin/2015-M-46813.tsv
../fulltext/jyu-theses/fin/2016-M-53917.tsv
../fulltext/jyu-theses/fin/2012-M-38182.tsv
../fulltext/jyu-theses/fin/2016-M-50632.tsv
../fulltext/jyu-theses/fin/2013-

...

../fulltext/jyu-theses/fin/2014-M-43651.tsv
../fulltext/jyu-theses/fin/2016-M-50619.tsv
../fulltext/jyu-theses/fin/2013-M-41794.tsv
../fulltext/jyu-theses/fin/2017-M-56235.tsv
../fulltext/jyu-theses/fin/2016-M-50207.tsv
../fulltext/jyu-theses/fin/2014-M-44696.tsv
../fulltext/jyu-theses/fin/2015-M-45646.tsv
../fulltext/jyu-theses/fin/2012-M-37973.tsv
../fulltext/jyu-theses/fin/2016-M-52167.tsv
../fulltext/jyu-theses/fin/2012-M-38118.tsv
../fulltext/jyu-theses/fin/2014-M-43644.tsv
../fulltext/jyu-theses/fin/2012-M-39953.tsv
../fulltext/jyu-theses/fin/2013-M-41949.tsv
../fulltext/jyu-theses/fin/2016-M-50591.tsv
../fulltext/jyu-theses/fin/2013-M-41994.tsv
../fulltext/jyu-theses/fin/2010-M-25658.tsv
../fulltext/jyu-theses/fin/2016-M-49309.tsv
../fulltext/jyu-theses/fin/2014-M-43970.tsv
../fulltext/jyu-theses/fin/2012-M-40421.tsv
../fulltext/jyu-theses/fin/2012-M-38352.tsv
../fulltext/jyu-theses/fin/2015-M-45582.tsv
../fulltext/jyu-theses/fin/2013-M-42659.tsv
../fulltext/jyu-theses/fin/2016-

../fulltext/jyu-theses/fin/2017-M-53220.tsv
../fulltext/jyu-theses/fin/2017-M-55773.tsv
../fulltext/jyu-theses/fin/2015-M-46604.tsv
../fulltext/jyu-theses/fin/2014-M-42900.tsv
../fulltext/jyu-theses/fin/2016-D-51813.tsv
../fulltext/jyu-theses/fin/2013-M-41804.tsv
../fulltext/jyu-theses/fin/2014-M-43427.tsv
../fulltext/jyu-theses/fin/2012-D-40442.tsv
../fulltext/jyu-theses/fin/2015-M-45806.tsv
../fulltext/jyu-theses/fin/2012-M-39965.tsv
../fulltext/jyu-theses/fin/2013-M-41105.tsv
../fulltext/jyu-theses/fin/2010-M-24645.tsv
../fulltext/jyu-theses/fin/2016-M-51873.tsv
../fulltext/jyu-theses/fin/2014-M-44760.tsv
../fulltext/jyu-theses/fin/2016-M-49865.tsv
../fulltext/jyu-theses/fin/2014-M-44978.tsv
../fulltext/jyu-theses/fin/2012-M-37951.tsv
../fulltext/jyu-theses/fin/2011-M-36583.tsv
../fulltext/jyu-theses/fin/2011-M-27204.tsv
../fulltext/jyu-theses/fin/2010-M-36728.tsv
../fulltext/jyu-theses/fin/2013-M-42473.tsv
../fulltext/jyu-theses/fin/2016-M-49969.tsv
../fulltext/jyu-theses/fin/2017-

../fulltext/jyu-theses/fin/2015-M-47328.tsv
../fulltext/jyu-theses/fin/2015-M-45697.tsv
../fulltext/jyu-theses/fin/2013-M-42680.tsv
../fulltext/jyu-theses/fin/2017-M-53982.tsv
../fulltext/jyu-theses/fin/2015-M-46209.tsv
../fulltext/jyu-theses/fin/2010-M-23815.tsv
../fulltext/jyu-theses/fin/2012-M-38215.tsv
../fulltext/jyu-theses/fin/2016-M-51585.tsv
../fulltext/jyu-theses/fin/2016-M-51477.tsv
../fulltext/jyu-theses/fin/2015-M-46264.tsv
../fulltext/jyu-theses/fin/2015-M-45148.tsv
../fulltext/jyu-theses/fin/2015-M-46111.tsv
../fulltext/jyu-theses/fin/2011-M-38026.tsv
../fulltext/jyu-theses/fin/2012-M-40581.tsv
../fulltext/jyu-theses/fin/2015-M-46188.tsv
../fulltext/jyu-theses/fin/2017-M-53569.tsv
../fulltext/jyu-theses/fin/2015-M-47608.tsv
../fulltext/jyu-theses/fin/2013-M-41824.tsv
../fulltext/jyu-theses/fin/2016-M-54936.tsv
../fulltext/jyu-theses/fin/2014-M-43878.tsv
../fulltext/jyu-theses/fin/2015-M-46385.tsv
../fulltext/jyu-theses/fin/2015-M-45947.tsv
../fulltext/jyu-theses/fin/2015-

../fulltext/jyu-theses/fin/2014-M-44912.tsv
../fulltext/jyu-theses/fin/2010-M-25731.tsv
../fulltext/jyu-theses/fin/2010-M-23890.tsv
../fulltext/jyu-theses/fin/2011-M-36873.tsv
../fulltext/jyu-theses/fin/2011-M-36997.tsv
../fulltext/jyu-theses/fin/2017-M-56908.tsv
../fulltext/jyu-theses/fin/2014-M-44733.tsv
../fulltext/jyu-theses/fin/2013-M-42042.tsv
../fulltext/jyu-theses/fin/2010-M-26539.tsv
../fulltext/jyu-theses/fin/2016-M-49971.tsv
../fulltext/jyu-theses/fin/2016-M-49024.tsv
../fulltext/jyu-theses/fin/2016-M-52273.tsv
../fulltext/jyu-theses/fin/2014-M-44979.tsv
../fulltext/jyu-theses/fin/2014-M-43010.tsv
../fulltext/jyu-theses/fin/2012-M-38044.tsv
../fulltext/jyu-theses/fin/2011-M-36788.tsv
../fulltext/jyu-theses/fin/2017-M-54521.tsv
../fulltext/jyu-theses/fin/2013-M-41263.tsv
../fulltext/jyu-theses/fin/2012-M-40831.tsv
../fulltext/jyu-theses/fin/2013-M-42668.tsv
../fulltext/jyu-theses/fin/2015-M-47841.tsv
../fulltext/jyu-theses/fin/2015-M-45248.tsv
../fulltext/jyu-theses/fin/2010-

../fulltext/jyu-theses/fin/2010-M-23250.tsv
../fulltext/jyu-theses/fin/2010-M-25551.tsv
../fulltext/jyu-theses/fin/2013-M-42830.tsv
../fulltext/jyu-theses/fin/2014-M-44829.tsv
../fulltext/jyu-theses/fin/2012-M-38090.tsv
../fulltext/jyu-theses/fin/2015-M-46405.tsv
../fulltext/jyu-theses/fin/2010-M-24460.tsv
../fulltext/jyu-theses/fin/2015-M-46048.tsv
../fulltext/jyu-theses/fin/2013-M-41891.tsv
../fulltext/jyu-theses/fin/2012-M-40410.tsv
../fulltext/jyu-theses/fin/2014-M-44476.tsv
../fulltext/jyu-theses/fin/2010-M-25669.tsv
../fulltext/jyu-theses/fin/2011-M-36560.tsv
../fulltext/jyu-theses/fin/2016-M-49669.tsv
../fulltext/jyu-theses/fin/2015-D-45611.tsv
../fulltext/jyu-theses/fin/2016-M-49974.tsv
../fulltext/jyu-theses/fin/2016-M-49585.tsv
../fulltext/jyu-theses/fin/2014-M-44536.tsv
../fulltext/jyu-theses/fin/2010-M-25514.tsv
../fulltext/jyu-theses/fin/2017-M-56542.tsv
../fulltext/jyu-theses/fin/2010-M-24422.tsv
../fulltext/jyu-theses/fin/2017-M-53365.tsv
../fulltext/jyu-theses/fin/2016-

In [6]:
sync_fulltext_dir('../fulltext/jyu-theses/swe/*.tsv', 'sv')

../fulltext/jyu-theses/swe/2011-M-26973.tsv
../fulltext/jyu-theses/swe/2012-M-38079.tsv
../fulltext/jyu-theses/swe/2013-M-42594.tsv
../fulltext/jyu-theses/swe/2013-M-41685.tsv
../fulltext/jyu-theses/swe/2015-M-46321.tsv
../fulltext/jyu-theses/swe/2016-M-49357.tsv
../fulltext/jyu-theses/swe/2012-M-38105.tsv
../fulltext/jyu-theses/swe/2013-M-41250.tsv
../fulltext/jyu-theses/swe/2012-M-37667.tsv
../fulltext/jyu-theses/swe/2015-M-45476.tsv
../fulltext/jyu-theses/swe/2016-M-52290.tsv
../fulltext/jyu-theses/swe/2014-M-43185.tsv
../fulltext/jyu-theses/swe/2012-M-38116.tsv
../fulltext/jyu-theses/swe/2011-M-26500.tsv
../fulltext/jyu-theses/swe/2013-M-42616.tsv
../fulltext/jyu-theses/swe/2011-M-37061.tsv
../fulltext/jyu-theses/swe/2010-M-24881.tsv
../fulltext/jyu-theses/swe/2013-M-42776.tsv
../fulltext/jyu-theses/swe/2011-M-37058.tsv
../fulltext/jyu-theses/swe/2012-M-38346.tsv
../fulltext/jyu-theses/swe/2015-M-47188.tsv
../fulltext/jyu-theses/swe/2012-M-40582.tsv
../fulltext/jyu-theses/swe/2013-

In [7]:
sync_fulltext_dir('../fulltext/jyu-theses/eng/*.tsv', 'en')

../fulltext/jyu-theses/eng/2012-M-40558.tsv
../fulltext/jyu-theses/eng/2017-M-52760.tsv
../fulltext/jyu-theses/eng/2012-D-37356.tsv
../fulltext/jyu-theses/eng/2016-M-51721.tsv
../fulltext/jyu-theses/eng/2011-D-56015.tsv
../fulltext/jyu-theses/eng/2014-M-43689.tsv
../fulltext/jyu-theses/eng/2013-M-42675.tsv
../fulltext/jyu-theses/eng/2016-M-52487.tsv
../fulltext/jyu-theses/eng/2014-M-44219.tsv
../fulltext/jyu-theses/eng/2014-M-44134.tsv
../fulltext/jyu-theses/eng/2014-M-44354.tsv
../fulltext/jyu-theses/eng/2013-D-42647.tsv
../fulltext/jyu-theses/eng/2013-M-42646.tsv
../fulltext/jyu-theses/eng/2014-M-43162.tsv
../fulltext/jyu-theses/eng/2016-D-49724.tsv
../fulltext/jyu-theses/eng/2014-D-42712.tsv
../fulltext/jyu-theses/eng/2012-M-37719.tsv
../fulltext/jyu-theses/eng/2010-D-24347.tsv
../fulltext/jyu-theses/eng/2011-M-37100.tsv
../fulltext/jyu-theses/eng/2010-D-25461.tsv
../fulltext/jyu-theses/eng/2017-M-54190.tsv
../fulltext/jyu-theses/eng/2014-D-44262.tsv
../fulltext/jyu-theses/eng/2012-

../fulltext/jyu-theses/eng/2015-M-48042.tsv
../fulltext/jyu-theses/eng/2016-D-51839.tsv
../fulltext/jyu-theses/eng/2014-M-44326.tsv
../fulltext/jyu-theses/eng/2017-M-55136.tsv
../fulltext/jyu-theses/eng/2014-M-44014.tsv
../fulltext/jyu-theses/eng/2015-M-45629.tsv
../fulltext/jyu-theses/eng/2013-M-42054.tsv
../fulltext/jyu-theses/eng/2016-M-49916.tsv
../fulltext/jyu-theses/eng/2017-M-53679.tsv
../fulltext/jyu-theses/eng/2013-D-42178.tsv
../fulltext/jyu-theses/eng/2014-D-44283.tsv
../fulltext/jyu-theses/eng/2016-D-49957.tsv
../fulltext/jyu-theses/eng/2017-M-53636.tsv
../fulltext/jyu-theses/eng/2012-M-37646.tsv
../fulltext/jyu-theses/eng/2017-M-53799.tsv
../fulltext/jyu-theses/eng/2014-D-44086.tsv
../fulltext/jyu-theses/eng/2015-D-47840.tsv
../fulltext/jyu-theses/eng/2015-D-46155.tsv
../fulltext/jyu-theses/eng/2016-M-52275.tsv
../fulltext/jyu-theses/eng/2013-D-41658.tsv
../fulltext/jyu-theses/eng/2013-M-42765.tsv
../fulltext/jyu-theses/eng/2017-M-56347.tsv
../fulltext/jyu-theses/eng/2014-

../fulltext/jyu-theses/eng/2014-D-42947.tsv
../fulltext/jyu-theses/eng/2016-M-52569.tsv
../fulltext/jyu-theses/eng/2012-M-40625.tsv
../fulltext/jyu-theses/eng/2014-M-43684.tsv
../fulltext/jyu-theses/eng/2014-M-44391.tsv
../fulltext/jyu-theses/eng/2015-D-47324.tsv
../fulltext/jyu-theses/eng/2015-D-49043.tsv
../fulltext/jyu-theses/eng/2015-D-45650.tsv
../fulltext/jyu-theses/eng/2014-D-44575.tsv
../fulltext/jyu-theses/eng/2017-M-56744.tsv
../fulltext/jyu-theses/eng/2014-M-45157.tsv
../fulltext/jyu-theses/eng/2017-M-55639.tsv
../fulltext/jyu-theses/eng/2013-D-42070.tsv
../fulltext/jyu-theses/eng/2011-M-26822.tsv
../fulltext/jyu-theses/eng/2016-M-50119.tsv
../fulltext/jyu-theses/eng/2017-M-55061.tsv
../fulltext/jyu-theses/eng/2015-M-45279.tsv
../fulltext/jyu-theses/eng/2010-M-25204.tsv
../fulltext/jyu-theses/eng/2014-D-44605.tsv
../fulltext/jyu-theses/eng/2014-M-43766.tsv
../fulltext/jyu-theses/eng/2011-M-37260.tsv
../fulltext/jyu-theses/eng/2011-D-37204.tsv
../fulltext/jyu-theses/eng/2015-

../fulltext/jyu-theses/eng/2014-M-44190.tsv
../fulltext/jyu-theses/eng/2015-M-52472.tsv
../fulltext/jyu-theses/eng/2016-D-52201.tsv
../fulltext/jyu-theses/eng/2016-M-50013.tsv
../fulltext/jyu-theses/eng/2011-M-26608.tsv
../fulltext/jyu-theses/eng/2013-M-41344.tsv
../fulltext/jyu-theses/eng/2014-D-44284.tsv
../fulltext/jyu-theses/eng/2015-M-45765.tsv
../fulltext/jyu-theses/eng/2016-D-52075.tsv
../fulltext/jyu-theses/eng/2015-M-47973.tsv
../fulltext/jyu-theses/eng/2012-M-42698.tsv
../fulltext/jyu-theses/eng/2015-D-46020.tsv
../fulltext/jyu-theses/eng/2011-D-36892.tsv
../fulltext/jyu-theses/eng/2013-D-42470.tsv
../fulltext/jyu-theses/eng/2014-M-44491.tsv
../fulltext/jyu-theses/eng/2014-M-43121.tsv
../fulltext/jyu-theses/eng/2012-D-40455.tsv
../fulltext/jyu-theses/eng/2017-M-54664.tsv
../fulltext/jyu-theses/eng/2016-D-49351.tsv
../fulltext/jyu-theses/eng/2016-M-52266.tsv
../fulltext/jyu-theses/eng/2014-D-44513.tsv
../fulltext/jyu-theses/eng/2011-M-26931.tsv
../fulltext/jyu-theses/eng/2013-

../fulltext/jyu-theses/eng/2010-M-22966.tsv
../fulltext/jyu-theses/eng/2012-D-40561.tsv
../fulltext/jyu-theses/eng/2016-M-52756.tsv
../fulltext/jyu-theses/eng/2012-M-40690.tsv
../fulltext/jyu-theses/eng/2016-M-49964.tsv
../fulltext/jyu-theses/eng/2014-M-43855.tsv
../fulltext/jyu-theses/eng/2013-M-42502.tsv
../fulltext/jyu-theses/eng/2015-D-48010.tsv
../fulltext/jyu-theses/eng/2016-M-50708.tsv
../fulltext/jyu-theses/eng/2013-M-42460.tsv
../fulltext/jyu-theses/eng/2016-M-51175.tsv
../fulltext/jyu-theses/eng/2015-D-47798.tsv
../fulltext/jyu-theses/eng/2016-M-50637.tsv
../fulltext/jyu-theses/eng/2010-M-25706.tsv
../fulltext/jyu-theses/eng/2015-D-45423.tsv
../fulltext/jyu-theses/eng/2016-D-50959.tsv
../fulltext/jyu-theses/eng/2013-D-42352.tsv
../fulltext/jyu-theses/eng/2011-D-36940.tsv
../fulltext/jyu-theses/eng/2017-D-55570.tsv
../fulltext/jyu-theses/eng/2012-M-38449.tsv
../fulltext/jyu-theses/eng/2017-D-54965.tsv
../fulltext/jyu-theses/eng/2017-M-54895.tsv
../fulltext/jyu-theses/eng/2017-

../fulltext/jyu-theses/eng/2015-M-46562.tsv
../fulltext/jyu-theses/eng/2011-D-37091.tsv
../fulltext/jyu-theses/eng/2017-M-53725.tsv
../fulltext/jyu-theses/eng/2017-M-54407.tsv
../fulltext/jyu-theses/eng/2010-M-23649.tsv
../fulltext/jyu-theses/eng/2015-D-47881.tsv
../fulltext/jyu-theses/eng/2017-M-54880.tsv
../fulltext/jyu-theses/eng/2016-M-50104.tsv
../fulltext/jyu-theses/eng/2017-M-54649.tsv
../fulltext/jyu-theses/eng/2015-M-47206.tsv
../fulltext/jyu-theses/eng/2014-M-45192.tsv
../fulltext/jyu-theses/eng/2012-M-38226.tsv
../fulltext/jyu-theses/eng/2013-M-42728.tsv
../fulltext/jyu-theses/eng/2015-M-50895.tsv
../fulltext/jyu-theses/eng/2014-M-43553.tsv
../fulltext/jyu-theses/eng/2014-M-44899.tsv
../fulltext/jyu-theses/eng/2011-M-36765.tsv
../fulltext/jyu-theses/eng/2014-D-44382.tsv
../fulltext/jyu-theses/eng/2017-D-52866.tsv
../fulltext/jyu-theses/eng/2017-D-52654.tsv
../fulltext/jyu-theses/eng/2012-D-40657.tsv
../fulltext/jyu-theses/eng/2015-M-46367.tsv
../fulltext/jyu-theses/eng/2015-

../fulltext/jyu-theses/eng/2012-M-37983.tsv
../fulltext/jyu-theses/eng/2011-D-37173.tsv
../fulltext/jyu-theses/eng/2016-M-49545.tsv
../fulltext/jyu-theses/eng/2015-M-46755.tsv
../fulltext/jyu-theses/eng/2014-M-46453.tsv
../fulltext/jyu-theses/eng/2017-M-55100.tsv
../fulltext/jyu-theses/eng/2017-M-56640.tsv
../fulltext/jyu-theses/eng/2015-M-45751.tsv
../fulltext/jyu-theses/eng/2012-M-40542.tsv
../fulltext/jyu-theses/eng/2012-D-37320.tsv
../fulltext/jyu-theses/eng/2011-D-36582.tsv
../fulltext/jyu-theses/eng/2017-M-54469.tsv
../fulltext/jyu-theses/eng/2011-M-37097.tsv
../fulltext/jyu-theses/eng/2016-M-51059.tsv
../fulltext/jyu-theses/eng/2012-M-40443.tsv
../fulltext/jyu-theses/eng/2017-M-55357.tsv
../fulltext/jyu-theses/eng/2017-D-54395.tsv
../fulltext/jyu-theses/eng/2014-M-43456.tsv
../fulltext/jyu-theses/eng/2013-M-42000.tsv
../fulltext/jyu-theses/eng/2013-M-42707.tsv
../fulltext/jyu-theses/eng/2011-D-40877.tsv
../fulltext/jyu-theses/eng/2012-M-37855.tsv
../fulltext/jyu-theses/eng/2012-

../fulltext/jyu-theses/eng/2011-M-26917.tsv
../fulltext/jyu-theses/eng/2017-M-55224.tsv
../fulltext/jyu-theses/eng/2012-M-37877.tsv
../fulltext/jyu-theses/eng/2015-M-46258.tsv
../fulltext/jyu-theses/eng/2012-D-40341.tsv
../fulltext/jyu-theses/eng/2011-M-36544.tsv
../fulltext/jyu-theses/eng/2014-M-43695.tsv
../fulltext/jyu-theses/eng/2016-D-48877.tsv
../fulltext/jyu-theses/eng/2013-D-42163.tsv
../fulltext/jyu-theses/eng/2017-M-55641.tsv
../fulltext/jyu-theses/eng/2014-D-43478.tsv
../fulltext/jyu-theses/eng/2017-M-54638.tsv
../fulltext/jyu-theses/eng/2017-D-53615.tsv
../fulltext/jyu-theses/eng/2016-M-51700.tsv
../fulltext/jyu-theses/eng/2016-M-50973.tsv
../fulltext/jyu-theses/eng/2011-M-36523.tsv
../fulltext/jyu-theses/eng/2012-D-38634.tsv
../fulltext/jyu-theses/eng/2017-M-53440.tsv
../fulltext/jyu-theses/eng/2011-M-36755.tsv
../fulltext/jyu-theses/eng/2015-M-46039.tsv
../fulltext/jyu-theses/eng/2012-M-40450.tsv
../fulltext/jyu-theses/eng/2012-D-40446.tsv
../fulltext/jyu-theses/eng/2012-

In [8]:
sync_fulltext_dir('../fulltext/arto-fi/all/*.tsv', 'fi')

../fulltext/arto-fi/all/1445895.tsv
../fulltext/arto-fi/all/1285213.tsv
../fulltext/arto-fi/all/1381728.tsv
../fulltext/arto-fi/all/1642477.tsv
../fulltext/arto-fi/all/535766.tsv
../fulltext/arto-fi/all/1724494.tsv
../fulltext/arto-fi/all/1641575.tsv
../fulltext/arto-fi/all/536577.tsv
../fulltext/arto-fi/all/1283036.tsv
../fulltext/arto-fi/all/535790.tsv
../fulltext/arto-fi/all/1180711.tsv
../fulltext/arto-fi/all/1339433.tsv
../fulltext/arto-fi/all/534079.tsv
../fulltext/arto-fi/all/1010545.tsv
../fulltext/arto-fi/all/532429.tsv
../fulltext/arto-fi/all/1193417.tsv
../fulltext/arto-fi/all/532419.tsv
../fulltext/arto-fi/all/863839.tsv
../fulltext/arto-fi/all/1029068.tsv
../fulltext/arto-fi/all/1642397.tsv
../fulltext/arto-fi/all/709454.tsv
../fulltext/arto-fi/all/1731194.tsv
../fulltext/arto-fi/all/1271671.tsv
../fulltext/arto-fi/all/1727345.tsv
../fulltext/arto-fi/all/1641179.tsv
../fulltext/arto-fi/all/1622050.tsv
../fulltext/arto-fi/all/536542.tsv
../fulltext/arto-fi/all/1381739.tsv
.

../fulltext/arto-fi/all/1331209.tsv
../fulltext/arto-fi/all/1066434.tsv
../fulltext/arto-fi/all/1339992.tsv
../fulltext/arto-fi/all/820539.tsv
../fulltext/arto-fi/all/1382458.tsv
../fulltext/arto-fi/all/1010632.tsv
../fulltext/arto-fi/all/986285.tsv
../fulltext/arto-fi/all/1378014.tsv
../fulltext/arto-fi/all/1206223.tsv
../fulltext/arto-fi/all/1190180.tsv
../fulltext/arto-fi/all/1368115.tsv
../fulltext/arto-fi/all/1085204.tsv
../fulltext/arto-fi/all/1381748.tsv
../fulltext/arto-fi/all/1355643.tsv
../fulltext/arto-fi/all/1381406.tsv
../fulltext/arto-fi/all/1641637.tsv
../fulltext/arto-fi/all/1376005.tsv
../fulltext/arto-fi/all/971336.tsv
../fulltext/arto-fi/all/1565806.tsv
../fulltext/arto-fi/all/1605791.tsv
../fulltext/arto-fi/all/1285235.tsv
../fulltext/arto-fi/all/1370922.tsv
../fulltext/arto-fi/all/1284499.tsv
../fulltext/arto-fi/all/1724691.tsv
../fulltext/arto-fi/all/1344687.tsv
../fulltext/arto-fi/all/532004.tsv
../fulltext/arto-fi/all/536358.tsv
../fulltext/arto-fi/all/1372465.t

../fulltext/arto-fi/all/1006485.tsv
../fulltext/arto-fi/all/1381702.tsv
../fulltext/arto-fi/all/1599166.tsv
../fulltext/arto-fi/all/960880.tsv
../fulltext/arto-fi/all/798898.tsv
../fulltext/arto-fi/all/531653.tsv
../fulltext/arto-fi/all/1375477.tsv
../fulltext/arto-fi/all/536478.tsv
../fulltext/arto-fi/all/1228063.tsv
../fulltext/arto-fi/all/1375482.tsv
../fulltext/arto-fi/all/818299.tsv
../fulltext/arto-fi/all/1001782.tsv
../fulltext/arto-fi/all/536581.tsv
../fulltext/arto-fi/all/1495211.tsv
../fulltext/arto-fi/all/1382177.tsv
../fulltext/arto-fi/all/1381852.tsv
../fulltext/arto-fi/all/1317779.tsv
../fulltext/arto-fi/all/1041142.tsv
../fulltext/arto-fi/all/531930.tsv
../fulltext/arto-fi/all/1353662.tsv
../fulltext/arto-fi/all/1523506.tsv
../fulltext/arto-fi/all/1036746.tsv
../fulltext/arto-fi/all/1642365.tsv
../fulltext/arto-fi/all/1616479.tsv
../fulltext/arto-fi/all/1004464.tsv
../fulltext/arto-fi/all/1193561.tsv
../fulltext/arto-fi/all/1615391.tsv
../fulltext/arto-fi/all/1370809.tsv

../fulltext/arto-fi/all/1641817.tsv
../fulltext/arto-fi/all/530589.tsv
../fulltext/arto-fi/all/1365542.tsv
../fulltext/arto-fi/all/535879.tsv
../fulltext/arto-fi/all/1358778.tsv
../fulltext/arto-fi/all/1206217.tsv
../fulltext/arto-fi/all/1693243.tsv
../fulltext/arto-fi/all/1057234.tsv
../fulltext/arto-fi/all/1643148.tsv
../fulltext/arto-fi/all/971501.tsv
../fulltext/arto-fi/all/1251454.tsv
../fulltext/arto-fi/all/1344629.tsv
../fulltext/arto-fi/all/1731129.tsv
../fulltext/arto-fi/all/1331852.tsv
../fulltext/arto-fi/all/781178.tsv
../fulltext/arto-fi/all/1039031.tsv
../fulltext/arto-fi/all/1279682.tsv
../fulltext/arto-fi/all/1028300.tsv
../fulltext/arto-fi/all/1502249.tsv
../fulltext/arto-fi/all/1641603.tsv
../fulltext/arto-fi/all/536367.tsv
../fulltext/arto-fi/all/1361500.tsv
../fulltext/arto-fi/all/1610147.tsv
../fulltext/arto-fi/all/1364301.tsv
../fulltext/arto-fi/all/1028521.tsv
../fulltext/arto-fi/all/1352365.tsv
../fulltext/arto-fi/all/1193224.tsv
../fulltext/arto-fi/all/1726469.t

../fulltext/arto-fi/all/1230062.tsv
../fulltext/arto-fi/all/1641648.tsv
../fulltext/arto-fi/all/1641732.tsv
../fulltext/arto-fi/all/1377890.tsv
../fulltext/arto-fi/all/821339.tsv
../fulltext/arto-fi/all/1724679.tsv
../fulltext/arto-fi/all/1612155.tsv
../fulltext/arto-fi/all/1047320.tsv
../fulltext/arto-fi/all/1359873.tsv
../fulltext/arto-fi/all/1250939.tsv
../fulltext/arto-fi/all/1641779.tsv
../fulltext/arto-fi/all/1495216.tsv
../fulltext/arto-fi/all/1377974.tsv
../fulltext/arto-fi/all/1642375.tsv
../fulltext/arto-fi/all/1546316.tsv
../fulltext/arto-fi/all/919568.tsv
../fulltext/arto-fi/all/1616612.tsv
../fulltext/arto-fi/all/1616820.tsv
../fulltext/arto-fi/all/1364591.tsv
../fulltext/arto-fi/all/1223134.tsv
../fulltext/arto-fi/all/998339.tsv
../fulltext/arto-fi/all/1693281.tsv
../fulltext/arto-fi/all/533482.tsv
../fulltext/arto-fi/all/532422.tsv
../fulltext/arto-fi/all/1022628.tsv
../fulltext/arto-fi/all/1357956.tsv
../fulltext/arto-fi/all/530587.tsv
../fulltext/arto-fi/all/1282239.ts

...

../fulltext/arto-fi/all/1373594.tsv
../fulltext/arto-fi/all/1028713.tsv
../fulltext/arto-fi/all/711202.tsv
../fulltext/arto-fi/all/535865.tsv
../fulltext/arto-fi/all/878809.tsv
../fulltext/arto-fi/all/878536.tsv
../fulltext/arto-fi/all/1206018.tsv
../fulltext/arto-fi/all/1727055.tsv
../fulltext/arto-fi/all/1201337.tsv
../fulltext/arto-fi/all/1187553.tsv
../fulltext/arto-fi/all/1360118.tsv
../fulltext/arto-fi/all/1342882.tsv
../fulltext/arto-fi/all/1726365.tsv
../fulltext/arto-fi/all/1380336.tsv
../fulltext/arto-fi/all/531972.tsv
../fulltext/arto-fi/all/1374124.tsv
../fulltext/arto-fi/all/1000291.tsv
../fulltext/arto-fi/all/1732078.tsv
../fulltext/arto-fi/all/1002714.tsv
../fulltext/arto-fi/all/1084235.tsv
../fulltext/arto-fi/all/531577.tsv
../fulltext/arto-fi/all/1725420.tsv
../fulltext/arto-fi/all/960157.tsv
../fulltext/arto-fi/all/1641939.tsv
../fulltext/arto-fi/all/1353616.tsv
../fulltext/arto-fi/all/996701.tsv
../fulltext/arto-fi/all/532475.tsv
../fulltext/arto-fi/all/1641727.tsv
.

../fulltext/arto-fi/all/971279.tsv
../fulltext/arto-fi/all/832727.tsv
../fulltext/arto-fi/all/976691.tsv
../fulltext/arto-fi/all/1643183.tsv
../fulltext/arto-fi/all/1264831.tsv
../fulltext/arto-fi/all/1228112.tsv
../fulltext/arto-fi/all/1641917.tsv
../fulltext/arto-fi/all/1617107.tsv
../fulltext/arto-fi/all/1279676.tsv
../fulltext/arto-fi/all/1373783.tsv
../fulltext/arto-fi/all/860369.tsv
../fulltext/arto-fi/all/1221426.tsv
../fulltext/arto-fi/all/1726475.tsv
../fulltext/arto-fi/all/1367189.tsv
../fulltext/arto-fi/all/1193341.tsv
../fulltext/arto-fi/all/1361444.tsv
../fulltext/arto-fi/all/1602703.tsv
../fulltext/arto-fi/all/531817.tsv
../fulltext/arto-fi/all/1022533.tsv
../fulltext/arto-fi/all/1296108.tsv
../fulltext/arto-fi/all/535775.tsv
../fulltext/arto-fi/all/1006036.tsv
../fulltext/arto-fi/all/859725.tsv
../fulltext/arto-fi/all/863773.tsv
../fulltext/arto-fi/all/711203.tsv
../fulltext/arto-fi/all/1285426.tsv
../fulltext/arto-fi/all/1206115.tsv
../fulltext/arto-fi/all/1611734.tsv
.

../fulltext/arto-fi/all/1030641.tsv
../fulltext/arto-fi/all/1578786.tsv
../fulltext/arto-fi/all/961591.tsv
../fulltext/arto-fi/all/1693438.tsv
../fulltext/arto-fi/all/1279554.tsv
../fulltext/arto-fi/all/531056.tsv
../fulltext/arto-fi/all/1364473.tsv
../fulltext/arto-fi/all/1612587.tsv
../fulltext/arto-fi/all/1028452.tsv
../fulltext/arto-fi/all/970885.tsv
../fulltext/arto-fi/all/1618690.tsv
../fulltext/arto-fi/all/536533.tsv
../fulltext/arto-fi/all/1378818.tsv
../fulltext/arto-fi/all/1022468.tsv
../fulltext/arto-fi/all/965622.tsv
../fulltext/arto-fi/all/1188170.tsv
../fulltext/arto-fi/all/1288257.tsv
../fulltext/arto-fi/all/536218.tsv
../fulltext/arto-fi/all/968800.tsv
../fulltext/arto-fi/all/1285439.tsv
../fulltext/arto-fi/all/1583969.tsv
../fulltext/arto-fi/all/1612413.tsv
../fulltext/arto-fi/all/1034262.tsv
../fulltext/arto-fi/all/1725697.tsv
../fulltext/arto-fi/all/1376289.tsv
../fulltext/arto-fi/all/1028063.tsv
../fulltext/arto-fi/all/1036788.tsv
../fulltext/arto-fi/all/1642868.tsv

../fulltext/arto-fi/all/1221170.tsv
../fulltext/arto-fi/all/1037226.tsv
../fulltext/arto-fi/all/807080.tsv
../fulltext/arto-fi/all/986648.tsv
../fulltext/arto-fi/all/1279814.tsv
../fulltext/arto-fi/all/1364738.tsv
../fulltext/arto-fi/all/1558790.tsv
../fulltext/arto-fi/all/1250756.tsv
../fulltext/arto-fi/all/885836.tsv
../fulltext/arto-fi/all/1283019.tsv
../fulltext/arto-fi/all/1357918.tsv
../fulltext/arto-fi/all/534098.tsv
../fulltext/arto-fi/all/1375264.tsv
../fulltext/arto-fi/all/1599348.tsv
../fulltext/arto-fi/all/709450.tsv
../fulltext/arto-fi/all/1381764.tsv
../fulltext/arto-fi/all/1187306.tsv
../fulltext/arto-fi/all/1550226.tsv
../fulltext/arto-fi/all/1022508.tsv
../fulltext/arto-fi/all/1726858.tsv
../fulltext/arto-fi/all/1383763.tsv
../fulltext/arto-fi/all/535706.tsv
../fulltext/arto-fi/all/1642858.tsv
../fulltext/arto-fi/all/1380074.tsv
../fulltext/arto-fi/all/1364355.tsv
../fulltext/arto-fi/all/1284117.tsv
../fulltext/arto-fi/all/878202.tsv
../fulltext/arto-fi/all/1382550.tsv

../fulltext/arto-fi/all/1085821.tsv
../fulltext/arto-fi/all/1366376.tsv
../fulltext/arto-fi/all/1361075.tsv
../fulltext/arto-fi/all/1357835.tsv
../fulltext/arto-fi/all/859758.tsv
../fulltext/arto-fi/all/535854.tsv
../fulltext/arto-fi/all/1332592.tsv
../fulltext/arto-fi/all/1375715.tsv
../fulltext/arto-fi/all/1042221.tsv
../fulltext/arto-fi/all/1193491.tsv
../fulltext/arto-fi/all/1381163.tsv
../fulltext/arto-fi/all/1725638.tsv
../fulltext/arto-fi/all/1187300.tsv
../fulltext/arto-fi/all/1248831.tsv
../fulltext/arto-fi/all/259473.tsv
../fulltext/arto-fi/all/1028729.tsv
../fulltext/arto-fi/all/1331829.tsv
../fulltext/arto-fi/all/532438.tsv
../fulltext/arto-fi/all/862670.tsv
../fulltext/arto-fi/all/1373333.tsv
../fulltext/arto-fi/all/1221328.tsv
../fulltext/arto-fi/all/1349736.tsv
../fulltext/arto-fi/all/1355649.tsv
../fulltext/arto-fi/all/1601512.tsv
../fulltext/arto-fi/all/1378381.tsv
../fulltext/arto-fi/all/1618643.tsv
../fulltext/arto-fi/all/1360981.tsv
../fulltext/arto-fi/all/1376097.t

In [46]:
# This function can be used to update files in the short text corpus format (single TSV file)

def read_short_tsv(fn):
    with open(fn) as tsvf:
        for line in tsvf:
            line = line.strip()
            if '\t' not in line:
                continue
            text, uris = line.split('\t', 1)
            uris = [uri[1:-1] for uri in uris.split()]
            yield (text, uris)

with open('yso-finna-new.tsv', 'w') as newf:
    for text, uris in read_short_tsv('yso-finna.tsv'):
        newuris = []
        for uri in uris:
            for newuri, label in replace_concept((uri, '')):
                if check_concept(newuri):
                    newuris.append(newuri)
        if newuris:
            print("{}\t{}".format(text, '\t'.join(("<{}>".format(uri) for uri in newuris))), file=newf)


http://www.yso.fi/onto/yso/p234587 not a skos:Concept
http://www.yso.fi/onto/yso/p24637 has no label
http://www.yso.fi/onto/yso/p5136 has no label
replacing http://www.yso.fi/onto/yso/p5195 with http://www.yso.fi/onto/yso/p8090
http://www.yso.fi/onto/yso/p23768 has no label
http://www.yso.fi/onto/yso/p234598 not a skos:Concept
http://www.yso.fi/onto/yso/p234564 not a skos:Concept
http://www.yso.fi/onto/yso/p234872 not a skos:Concept
http://www.yso.fi/onto/yso/p234554 not a skos:Concept
http://www.yso.fi/onto/yso/p22726 has no label
http://www.yso.fi/onto/yso/p506078 has no label
http://www.yso.fi/onto/yso/p506996 not a skos:Concept
http://www.yso.fi/onto/yso/p506096 has no label
http://www.yso.fi/onto/yso/p234581 not a skos:Concept
http://www.yso.fi/onto/yso/p220088 has no label
http://www.yso.fi/onto/yso/p234597 not a skos:Concept
http://www.yso.fi/onto/yso/p234577 not a skos:Concept
http://www.yso.fi/onto/yso/p105863 has no label
http://www.yso.fi/onto/yso/p234548 not a skos:Concept


http://www.yso.fi/onto/yso/p226831 has no label
http://www.yso.fi/onto/yso/p94421 has no label
http://www.yso.fi/onto/yso/p234609 not a skos:Concept
http://www.yso.fi/onto/yso/p234755 not a skos:Concept
http://www.yso.fi/onto/yso/p147448 has no label
http://www.yso.fi/onto/yso/p234616 not a skos:Concept
http://www.yso.fi/onto/yso/p234592 not a skos:Concept
http://www.yso.fi/onto/yso/p124440 has no label
http://www.yso.fi/onto/yso/p38388 not a skos:Concept
http://www.yso.fi/onto/yso/p110010 has no label
http://www.yso.fi/onto/yso/p139368 has no label
http://www.yso.fi/onto/yso/p174745 has no label
http://www.yso.fi/onto/yso/p38429 not a skos:Concept
http://www.yso.fi/onto/yso/p106243 has no label
http://www.yso.fi/onto/yso/p38018 has no label
http://www.yso.fi/onto/yso/p38444 not a skos:Concept
http://www.yso.fi/onto/yso/p227116 has no label
http://www.yso.fi/onto/yso/p38359 not a skos:Concept
http://www.yso.fi/onto/yso/p217601 has no label
http://www.yso.fi/onto/yso/p149181 has no labe

http://www.yso.fi/onto/yso/p226827 has no label
http://www.yso.fi/onto/yso/p219271 has no label
http://www.yso.fi/onto/yso/p506249 not a skos:Concept
http://www.yso.fi/onto/yso/p166486 has no label
http://www.yso.fi/onto/yso/p234956 not a skos:Concept
http://www.yso.fi/onto/yso/p38373 not a skos:Concept
http://www.yso.fi/onto/yso/p145978 has no label
http://www.yso.fi/onto/yso/p111001 has no label
http://www.yso.fi/onto/yso/p105084 has no label
http://www.yso.fi/onto/yso/p234580 not a skos:Concept
http://www.yso.fi/onto/yso/p113096 has no label
http://www.yso.fi/onto/yso/p110165 has no label
http://www.yso.fi/onto/yso/p506125 not a skos:Concept
http://www.yso.fi/onto/yso/p38357 not a skos:Concept
http://www.yso.fi/onto/yso/p215780 has no label
http://www.yso.fi/onto/yso/p507154 not a skos:Concept
http://www.yso.fi/onto/yso/p38309 not a skos:Concept
http://www.yso.fi/onto/yso/p173436 has no label
http://www.yso.fi/onto/yso/p38447 not a skos:Concept
http://www.yso.fi/onto/yso/p26514 not 